[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/notebook-share/blob/master/docs/pakistan_floods.ipynb)
[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/giswqs/notebook-share/blob/master/docs/pakistan_floods.ipynb)
[![Open in Planetary Computer](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/giswqs/notebook-share&urlpath=lab/tree/notebook-share/docs/pakistan_floods.ipynb&branch=master)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giswqs/notebook-share/HEAD?labpath=docs%pakistan_floods.ipynb)

# Peru Floods 2022 Using Earth Engine and Geemap - Landsat 8



## Import libraries

In [1]:
import ee
import geemap

In [2]:
pre_flood_start_date = '2022-09-01'
pre_flood_end_date = '2022-11-30'
flood_start_date = '2023-02-01'
flood_end_date = '2023-04-16'

## Create Landsat composites

Create a Landsat 8 composite for the pre-flood period in the dry season (September to November, 2022) and the flooding period (February to April) using the [USGS Landsat 8 Collection 2 Tier 1 Raw Scenes](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1).

In [3]:
Map = geemap.Map()

# Get Peru bounds
country_name = 'Peru'

country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_na', country_name)
)

# Pre-flood
landsat_col_2022 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1')
    .filterDate(pre_flood_start_date, pre_flood_end_date)
    .filterBounds(country)
)
landsat_2022 = ee.Algorithms.Landsat.simpleComposite(landsat_col_2022).clipToCollection(
    country
)

# Post-flood
landsat_col_2023 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1')
    .filterDate(flood_start_date, flood_end_date)
    .filterBounds(country)
)
landsat_2023 = ee.Algorithms.Landsat.simpleComposite(landsat_col_2023).clipToCollection(
    country
)

# Display on a map
style = {'color': 'black', 'fillColor': '00000000'}
Map.addLayer(country.style(**style), {}, country_name)

vis_params = {'bands': ['B6', 'B5', 'B4'], 'max': 128}
Map.addLayer(landsat_2022, vis_params, 'Landsat 2022')
Map.addLayer(landsat_2023, vis_params, 'Landsat 2023')

Map.centerObject(country)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Compute Normalized Difference Water Index (NDWI) and Modified NDWI (MNDWI)

The [Normalized Difference Water Index](https://en.wikipedia.org/wiki/Normalized_difference_water_index) (NDWI) is a commonly used index for detecting water bodies. It is calculated as follows:

$$NDWI = \frac{Green - NIR}{Green + NIR}$$

where Green is the green band and NIR is the near-infrared band. The NDWI values range from -1 to 1. The NDWI values are usually thresholded to a positive number (e.g., 0.1-0.3) to identify water bodies.

[MNDWI](https://www.space4water.org/taxonomy/term/1246) uses green and SWIR bands for the enhancement of open water features. It also diminishes built-up area features that are often correlated with open water in other indices.

Landsat 8 imagery has [11 spectral bands](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1#bands). The Landsat 8 NDWI is calculated using the green (`B3`) and NIR (`B5`) bands. The Landsat 8 MNDWI is calculated using the green (`B3`) and SWIR (`B6`) bands.

![](https://i.imgur.com/yuZthc6.png)

In [25]:
ndwi_2022 = landsat_2022.normalizedDifference(['B3', 'B5']).rename('NDWI')
ndwi_2023 = landsat_2023.normalizedDifference(['B3', 'B5']).rename('NDWI')

mndwi_2022 = landsat_2022.normalizedDifference(['B3', 'B6']).rename('MNDWI')
mndwi_2023 = landsat_2023.normalizedDifference(['B3', 'B6']).rename('MNDWI')

In [26]:
Map = geemap.Map()
Map.setCenter(-81.5, -3.6, 7)

ndwi_vis = {'min': -1, 'max': 1, 'palette': 'ndwi'}

Map.addLayer(ndwi_2022, ndwi_vis, 'NDWI 2022')
Map.addLayer(ndwi_2023, ndwi_vis, 'NDWI 2023')
Map.addLayer(mndwi_2022, ndwi_vis, 'MNDWI 2022')
Map.addLayer(mndwi_2023, ndwi_vis, 'MNDWI 2023')

Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[-3.6, -81.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

## Extract Landsat water extent

To extract the water extent, we need to convert the NDWI images to binary images using a threshold value. The threshold value is usually set to 0.1 to 0.3. The smaller the threshold value, the more water bodies will be detected, which may increase the false positive rate. The larger the threshold value, the fewer water bodies will be detected, which may increase the false negative rate.

In [31]:
ndwi_threshold = 0.3877 # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6111878/
ndwi_water_2022 = ndwi_2022.gt(ndwi_threshold)
ndwi_water_2023 = ndwi_2023.gt(ndwi_threshold)

mndwi_threshold = 0.35 # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6111878/
mndwi_water_2022 = mndwi_2022.gt(mndwi_threshold)
mndwi_water_2023 = mndwi_2023.gt(mndwi_threshold)

Combine the pre-flood and surface water extent side by side.

In [32]:
Map = geemap.Map()
Map.setCenter(-81.5, -3.6, 7)

Map.addLayer(landsat_2022, vis_params, 'Landsat 2022', False)
Map.addLayer(landsat_2023, vis_params, 'Landsat 2023', False)

Map.addLayer(ndwi_water_2022.selfMask(), {'palette': 'blue'}, 'NDWI Water 2022')
Map.addLayer(ndwi_water_2023.selfMask(), {'palette': 'red'}, 'NDWI Water  2023')
Map.addLayer(mndwi_water_2022.selfMask(), {'palette': 'yellow'}, 'MNDWI Water 2022')
Map.addLayer(mndwi_water_2023.selfMask(), {'palette': 'orange'}, 'MNDWI Water 2023')

Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[-3.6, -81.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

## Extract Landsat flood extent

To extract the flood extent, we need to subtract the pre-flood water extent from the flood water extent. The flood extent is the difference between the flood water extent and the pre-flood water extent. In other words, pixels identified as water in the flood period but not in the pre-flood period are considered as flooded pixels. The `selfMask()` method is used to mask out the no-data pixels.

In [33]:
ndwi_flood_extent = ndwi_water_2023.subtract(ndwi_water_2022).gt(0).selfMask()
mndwi_flood_extent = mndwi_water_2023.subtract(mndwi_water_2022).gt(0).selfMask()

Add the flood extent layer to the map.

In [37]:
Map = geemap.Map()
Map.setCenter(-81.5, -3.6, 7)

Map.addLayer(landsat_2022, vis_params, 'Landsat 2022', False)
Map.addLayer(landsat_2023, vis_params, 'Landsat 2023', False)

# Map.addLayer(ndwi_water_2022.selfMask(), {'palette': 'blue'}, 'NDWI Water 2022')
# Map.addLayer(ndwi_water_2023.selfMask(), {'palette': 'red'}, 'NDWI Water  2023')
# Map.addLayer(mndwi_water_2022.selfMask(), {'palette': 'yellow'}, 'MNDWI Water 2022')
# Map.addLayer(mndwi_water_2023.selfMask(), {'palette': 'orange'}, 'MNDWI Water 2023')


Map.addLayer(ndwi_flood_extent, {'palette': 'cyan'}, 'NDWI Flood Extent')
Map.addLayer(mndwi_flood_extent, {'palette': 'magenta'}, 'MNDWI Flood Extent')

Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[-3.6, -81.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

## Calculate Landsat flood area

To calculate the flood area, we can use the [`geemap.zonal_stats()`](https://geemap.org/common/#geemap.common.zonal_stats) function. The required input parameters are the flood extent layer and the country boundary layer. The `scale` parameter can be set to `1000` to specify the spatial resolution of image to be used for calculating the zonal statistics. The `stats_type` parameter can be set to `SUM` to calculate the total area of the flood extent in square kilometers. Set `return_fc=True` to return the zonal statistics as an `ee.FeatureCollection` object, which can be converted to a Pandas dataframe.

In [35]:
ndwi_flood_area = geemap.zonal_stats(
    ndwi_flood_extent.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(ndwi_flood_area)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,2316.592157,South America,Peru,,PE


NDWI - The total area of the flood extent is 2316 square kilometers based on Landsat 8 images.

In [36]:
mndwi_flood_area = geemap.zonal_stats(
    mndwi_flood_extent.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(mndwi_flood_area)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,40172.529412,South America,Peru,,PE


MNDWI - The total area of the flood extent is 40172 square kilometers based on Landsat 8 images.